In [ ]:
# %load train.py
#!/usr/bin/env python

# In[1]:
from t_cnn import tcnn

model = tcnn(num_classes = 2,pretrained = True, model_root = '/home/krf/model/BALL/')

import senet
import os
import numpy as np
import torch
from torchvision.datasets import ImageFolder
from utils import TransformImage
import shutil
import time

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tensorboardX import SummaryWriter

DATA_DIR = "/home/krf/dataset/BALL/"
traindir = DATA_DIR + "train"
valdir = DATA_DIR +"val"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
BATCH_SIZE = 32
WORKERS = 4
START = 0
EPOCHS = 1200
PRINT_FREQ = 20


# In[31]:


#model =  senet.se_resnext50_32x4d(num_classes = 2)
#通过随机变化来进行数据增强
train_tf  = TransformImage(
    model,
    
    random_crop=False,
    random_hflip=True,
    random_vflip=True,
    random_rotate=True,
    preserve_aspect_ratio=True
)
train_loader = torch.utils.data.DataLoader(
#     datasets.ImageFolder(traindir, transforms.Compose([
# #         transforms.RandomSizedCrop(max(model.input_size)),
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         normalize,
#     ])),
    datasets.ImageFolder(traindir,train_tf),
    batch_size=BATCH_SIZE, shuffle=True,
    num_workers=WORKERS, pin_memory=True)


val_tf = TransformImage(
    model,
    
    preserve_aspect_ratio=True)
val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(valdir,val_tf),
    batch_size=BATCH_SIZE, shuffle=False,
    num_workers=WORKERS, pin_memory=True)


# In[29]:


def train(train_loader, model, criterion, optimizer, epoch,scheduler):
    # switch to train mode
    model.train()
#     end = time.time()
    for i, (input, target) in enumerate(train_loader):
        # measure data loading time
#         data_time.update(time.time() - end)
        input = input.cuda()
        target = target.cuda(async=True)
        input_var = torch.autograd.Variable(input.float())
        target_var = torch.autograd.Variable(target)
        #print(input_var.type())
        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)
        #print(output.data)



#         # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

#         # measure elapsed time
#         batch_time.update(time.time() - end)
#         end = time.time()
        meters = trainMeter.update(output,target,loss,input.size(0))

        if i % PRINT_FREQ == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Loss {loss:.5f}\t'
                  'Acc {Acc:.5f}\t'
                  'Precision {P:.5f}\t'
                  'Recall {R:.5f}\t'
                  'F1 {F1:.5f}'.format(
                   epoch,i, len(train_loader), loss=meters[4],
                   Acc=meters[3],P=meters[0],R=meters[1],F1=meters[2]))
            
            step = epoch*len(train_loader) + i
            
            writer.add_scalar('TRAIN/Precision', meters[0], step)
            writer.add_scalar('TRAIN/Recall', meters[1], step)
            writer.add_scalar('TRAIN/F1', meters[2], step)
            writer.add_scalar('TRAIN/Acc', meters[3], step)
            writer.add_scalar('TRAIN/loss',meters[4], step)
            
    scheduler.step(meters[4])


def validate(val_loader, model, criterion,epoch):
    # switch to evaluate mode
    model.eval()
    
#     end = time.time()
    meters = []
    for i, (input, target) in enumerate(val_loader):
        target = target.cuda()
        input = input.cuda()
        input_var = torch.autograd.Variable(input, volatile=True)
        target_var = torch.autograd.Variable(target, volatile=True)
        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)
        meters = valMeter.update(output,target,loss,input.size(0))
        if i % PRINT_FREQ == 0:
            print('Test: [{0}/{1}]\t'
                  'Loss {loss:.5f}\t'
                  'Acc {Acc:.5f}\t'
                  'Precision {P:.5f}\t'
                  'Recall {R:.5f}\t'
                  'F1 {F1:.5f}'.format(
                   i, len(val_loader), loss=meters[4],
                   Acc=meters[3],P=meters[0],R=meters[1],F1=meters[2]))
            
            step = epoch * len(val_loader) + i
            writer.add_scalar('VAL/Precision', meters[0], step)
            writer.add_scalar('VAL/Recall', meters[1], step)
            writer.add_scalar('VAL/F1', meters[2], step)
            writer.add_scalar('VAL/Acc', meters[3], step)
            writer.add_scalar('VAL/loss',meters[4], step)
    print(' * Acc {Acc:.5f} F1 {F1:.5f}'
          .format(Acc=meters[3],F1=meters[2]))
    writer.add_scalar('VAL/EPOCH_F1', meters[2], epoch)
    return meters[2]


def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'tcnn5_model_best.pth.tar')

class ModelMeter(object):
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.losses = AverageMeter()
        self.top1 = AverageMeter()
        self.TP = 1e-8
        self.TN = 1e-8
        self.FN = 1e-8
        self.FP = 1e-8
        self.P=1e-8
        self.R=1e-8
        self.F1=1e-8
        self.Acc=1e-8

    def update(self, output,target,loss, n=1):
        _, pred = output.data.topk(1, 1, True, True)
        pred = pred.t()
#         print(pred,target.data)
        # TP    predict 和 label 同时为1
        self.TP += ((pred == 1) & (target.data == 1)).cpu().numpy().sum()
        # TN    predict 和 label 同时为0
        self.TN += ((pred == 0) & (target.data == 0)).cpu().numpy().sum()
        # FN    predict 0 label 1
        self.FN += ((pred == 0) & (target.data == 1)).cpu().numpy().sum()
        # FP    predict 1 label 0
        self.FP += ((pred == 1) & (target.data == 0)).cpu().numpy().sum()
#         print(self.TP,self.TN,self.FN,self.FP)
#         zes=torch.autograd.Variable(torch.zeros(target.size()).type(torch.cuda.LongTensor))#全0变量

#         ons=torch.autograd.Variable(torch.ones(target.size()).type(torch.cuda.LongTensor))#全1变量
#         print(zes,ons)
#         train_correct01 = ((pred==zes)&(target.data.squeeze(1)==ons)).sum()#原标签为1，预测为 0 的总数

#         train_correct10 = ((pred==ons)&(target.data.squeeze(1)==zes)).sum()#原标签为0，预测为1  的总数

#         train_correct11 = ((pred_y==ons)&(target.data.squeeze(1)==ons)).sum()
#         train_correct00 = ((pred_y==zes)&(target.data.squeeze(1)==zes)).sum()
#         self.FN += train_correct01.data[0]        

#         self.FP += train_correct10.data[0]

#         self.TP += train_correct11.data[0]  
#         self.TN += train_correct00.data[0]
      
        self.P = self.TP / (self.TP + self.FP)
        self.R = self.TP / (self.TP + self.FN)
        self.F1 = 2 * self.R * self.P / (self.R + self.P)
        
        self.Acc = (self.TP + self.TN) / (self.TP + self.TN + self.FP + self.FN)
        
        self.losses.update(loss.data[0],n)

        return [self.P,self.R,self.F1,self.Acc,self.losses.avg]

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 1e-8
        self.avg = 1e-8
        self.sum = 1e-8
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def adjust_learning_rate(lr,optimizer, epoch):
    if epoch >= 300 and epoch % 100 == 0 :
        lr /= 10
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        print("adjuct learning rate to {}".format(lr))
    return lr


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


# In[ ]:

# 加载模型，解决命名和维度不匹配问题,解决多个gpu并行
def load_state_keywise(model, model_path):
    model_dict = model.state_dict()
    
    print("=> loading checkpoint '{}'".format(model_path))
    checkpoint = torch.load(model_path,map_location='cpu')
    START = checkpoint['epoch']
    best_F1 = checkpoint['best_prec1']
    #model.load_state_dict(checkpoint['state_dict'])
    
    pretrained_dict = checkpoint['state_dict']#torch.load(model_path, map_location='cpu')
    key = list(pretrained_dict.keys())[0]
    # 1. filter out unnecessary keys
    # 1.1 multi-GPU ->CPU
    if (str(key).startswith('module.')):
        pretrained_dict = {k[7:]: v for k, v in pretrained_dict.items() if
                           k[7:] in model_dict and v.size() == model_dict[k[7:]].size()}
    else:
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if
                           k in model_dict and v.size() == model_dict[k].size()}
    # 2. overwrite entries in the existing state dict
    model_dict.update(pretrained_dict)
    # 3. load the new state dict
    model.load_state_dict(model_dict)
    print("start at epoch {}, best_f1={}".format(START,best_F1))
    return model,START,best_F1



criterion = nn.CrossEntropyLoss().cuda()
lr = 1e-4
optimizer = torch.optim.SGD(model.parameters(), lr = lr,momentum = 0.9,weight_decay=1e-6)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, verbose=True)
best_f1 = 0
model,START,best_f1 = load_state_keywise(model,'tcnn5_checkpoint-Copy1.pth.tar')
# model = model.cuda()
model = torch.nn.DataParallel(model).cuda()

# TP = 0,TN = 0,FN = 0, FP = 0
writer = SummaryWriter()

trainMeter = ModelMeter()
valMeter = ModelMeter()
for epoch in range(START,EPOCHS):
    # train for one epoch
    lr = optimizer.param_groups[0]['lr']
    writer.add_scalar('LR', lr, epoch)
    train(train_loader, model, criterion, optimizer, epoch, scheduler)
    # evaluate on validation set
    F1 = validate(val_loader, model, criterion,epoch)
    
    # remember best prec@1 and save checkpoint
    is_best = F1 > best_f1
    best_f1 = max(F1, best_f1)
    save_checkpoint({
        'epoch': epoch + 1,
        'arch': "T-CNN",
        'state_dict': model.state_dict(),
        'best_prec1': best_f1,
    }, is_best,filename='tcnn5_checkpoint.pth.tar')
# export scalar data to JSON for external processing
writer.export_scalars_to_json("./test.json")
writer.close()







TCNN_Alex(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
  )
  (conv4): Sequential(
    (0): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
  )
  (conv5): Sequential(
    (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=9216, out_features=4096, bias=True)
  (classifier): Sequential(
    (0):

/home/krf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:223: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch: [800][0/278]	Loss 0.00495	Acc 1.00000	Precision 1.00000	Recall 1.00000	F1 1.00000
Epoch: [800][20/278]	Loss 0.01283	Acc 0.99702	Precision 1.00000	Recall 0.99057	F1 0.99526
Epoch: [800][40/278]	Loss 0.01331	Acc 0.99466	Precision 0.99215	Recall 0.98956	F1 0.99085
Epoch: [800][60/278]	Loss 0.01365	Acc 0.99488	Precision 0.99310	Recall 0.98969	F1 0.99139
Epoch: [800][80/278]	Loss 0.01386	Acc 0.99460	Precision 0.99332	Recall 0.98803	F1 0.99067
Epoch: [800][100/278]	Loss 0.01491	Acc 0.99443	Precision 0.99353	Recall 0.98715	F1 0.99033
Epoch: [800][120/278]	Loss 0.01522	Acc 0.99406	Precision 0.99191	Recall 0.98748	F1 0.98969
Epoch: [800][140/278]	Loss 0.01513	Acc 0.99424	Precision 0.99175	Recall 0.98878	F1 0.99026
Epoch: [800][160/278]	Loss 0.01546	Acc 0.99457	Precision 0.99207	Recall 0.98946	F1 0.99077
Epoch: [800][180/278]	Loss 0.01508	Acc 0.99482	Precision 0.99292	Recall 0.98941	F1 0.99116
Epoch: [800][200/278]	Loss 0.01503	Acc 0.99487	Precision 0.99311	Recall 0.98945	F1 0.99128
Epoch

/home/krf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:138: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/krf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:139: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


Test: [0/21]	Loss 0.08331	Acc 0.96875	Precision 0.00000	Recall 0.50000	F1 0.00000
Test: [20/21]	Loss 0.60053	Acc 0.86165	Precision 0.89815	Recall 0.73485	F1 0.80833
 * Acc 0.86165 F1 0.80833
Epoch: [801][0/278]	Loss 0.01662	Acc 0.99462	Precision 0.99269	Recall 0.98889	F1 0.99079
Epoch: [801][20/278]	Loss 0.01618	Acc 0.99477	Precision 0.99250	Recall 0.98967	F1 0.99108
Epoch: [801][40/278]	Loss 0.01609	Acc 0.99500	Precision 0.99295	Recall 0.98995	F1 0.99145
Epoch: [801][60/278]	Loss 0.01553	Acc 0.99529	Precision 0.99336	Recall 0.99055	F1 0.99195
Epoch: [801][80/278]	Loss 0.01571	Acc 0.99530	Precision 0.99343	Recall 0.99047	F1 0.99195
Epoch: [801][100/278]	Loss 0.01522	Acc 0.99554	Precision 0.99377	Recall 0.99097	F1 0.99237
Epoch: [801][120/278]	Loss 0.01520	Acc 0.99561	Precision 0.99411	Recall 0.99092	F1 0.99251
Epoch: [801][140/278]	Loss 0.01511	Acc 0.99560	Precision 0.99387	Recall 0.99109	F1 0.99248
Epoch: [801][160/278]	Loss 0.01527	Acc 0.99544	Precision 0.99412	Recall 0.99024	F1 0.99

Epoch: [806][160/278]	Loss 0.01749	Acc 0.99437	Precision 0.99263	Recall 0.98818	F1 0.99040
Epoch: [806][180/278]	Loss 0.01750	Acc 0.99440	Precision 0.99265	Recall 0.98824	F1 0.99044
Epoch: [806][200/278]	Loss 0.01754	Acc 0.99438	Precision 0.99261	Recall 0.98819	F1 0.99040


In [6]:
# import torch.utils.model_zoo as model_zoo
# model_dict = model_zoo.load_url('https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth', '/home/krf/model/BALL/')

In [10]:
# print(model_dict.keys())

odict_keys(['features.0.weight', 'features.0.bias', 'features.3.weight', 'features.3.bias', 'features.6.weight', 'features.6.bias', 'features.8.weight', 'features.8.bias', 'features.10.weight', 'features.10.bias', 'classifier.1.weight', 'classifier.1.bias', 'classifier.4.weight', 'classifier.4.bias', 'classifier.6.weight', 'classifier.6.bias'])


In [16]:
# model = tcnn(2)
# pre_dict = model.state_dict()
# print(pre_dict.keys())

AttributeError: 'odict_keys' object has no attribute 'values'

In [ ]:
# k1 = ['features.0.weight', 'features.0.bias', 'features.3.weight', 'features.3.bias', 'features.6.weight', 'features.6.bias', 'features.8.weight', 'features.8.bias', 'features.10.weight', 'features.10.bias', 'classifier.1.weight', 'classifier.1.bias', 'classifier.4.weight', 'classifier.4.bias', 'classifier.6.weight', 'classifier.6.bias']
# k2 = ['conv1.0.weight', 'conv1.0.bias', 'conv2.0.weight', 'conv2.0.bias', 'conv3.0.weight', 'conv3.0.bias', 'conv4.0.weight', 'conv4.0.bias', 'conv5.0.weight', 'conv5.0.bias', 'fc1.weight', 'fc1.bias', 'classifier.1.weight', 'classifier.1.bias', 'classifier.3.weight', 'classifier.3.bias']
# for i in range(len(k1)):
#     pretrained_dict[k2[i]] = model_dict[k1[i]]

# # 2. overwrite entries in the existing state dict
# model_dict.update(pretrained_dict)